In [4]:
import torch
import numpy as np
from torch import nn

## ScaledDotProductAttentionの実装
---
### 概要
---
qは***[batch,T,d_model]***の次元を持っている。
qはhで割られて分割されるので分割された後[h,batch,T,d_k]の次元になる。この時h×d_k=d_modelとなる。
このd_kを使ってスケーリングをする。


### attention_maskの実装
---
attention_weightに対して同じ形状のmaskは未来の情報を参照しないように上三角行列のようなマスクをかける。
tensor.masked_fill_()メソッドは元テンソルとmaskの形状が同じ時maskのTrueの部分に対して第二引数に与えられた値で元テンソルを埋める。
今回はその値がfloatで指定できる一番小さい値であるためsoftmaxをかけた時0に行くようになっている。


### logitの作成
---
これは見慣れたいつもの式でDotProductattentionを作成する。
```
torch.matmul(q,torch.transpose(k,1,2))/d_k**(0.5)
```


### softmaxを通す
---
これも見慣れた式でやる。
```
nn.functional.softmax(logit,dim=2)
```


### vと掛け算する。
---
これもいつも通り
```
torch.matmul(attention_weight,v)
```

In [5]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self,
                 d_k:int
                ) -> None:
        super().__init__()
        self.d_k = d_k
    def forward(
        self,
        q:torch.Tensor,
        k:torch.Tensor,
        v:torch.Tensor,
        mask:torch.Tensor = None,
    ) -> torch.Tensor:
        
        ###次元の調節
        scaler = self.d_k**(0.5)
        ###attention_weightの作成。
        logit = torch.matmul(q,torch.transpose(k,1,2))/scaler
        ###maskの作成
        ###maskがNoneじゃなかったらshapeの確認
        if mask is not None:
            if mask.dim() != attention_weight.dim():
                print("mask must have same dim with attention_weight")
            else:
                with torch.no_grad():
                    attention_weight = logit.masked_fill_(
                      mask,
                      -torch.finfo(torch.float).max
                    )
        attention_weight = nn.functional.softmax(logit,dim=2)
        return torch.matmul(attention_weight,v)